In [25]:
from collections import Counter
from itertools import combinations
from itertools import permutations
import time
from _md5 import md5
# Define global variables
ANAGRAM = "poultry outwits ants"
HASH_EASY = "e4820b45d2277f3844eac66c903e84be"
HASH_MED = "23170acc097c24edb98fc5488ab033fe"
HASH_HARD = "665e5bcb0c20062fe8abaaf4628bb154"
PHRASE_CHARACTERS = len(ANAGRAM) - ANAGRAM.count(' ')
PHRASE_COUNTER = Counter(ANAGRAM.replace(" ", ""))
# The replace is necessary when we'll be dealing with >3 words

# HELPER FUNCTIONS


def is_counter_subset(reference_counter, words_counter):
    r"""Function to define if a collections.Counter is a subset of the
    reference one.

    Parameters
    ----------
    reference_counter:    dictionary
                          ANAGRAM counter

    words_counter:        dictionary
                          current phrase or word counter

    Return
    ------
    is_subset:            Bool
                          True is a subset, False it is not

    """
    # Check if the counters are subset of anagram_letters
    is_subset_counter = Counter({
                                key: reference_counter.get(key, 0) - value
                                for key, value in words_counter.items()
                                })

    #  If characters are included in the reference counter keep this word
    if any(v < 0 for v in is_subset_counter.values()):
        # In this case a negative element is present for some letters
        is_subset = False
        return is_subset
    else:
        is_subset = True
        return is_subset


def permutations_to_string(curr_phrase, curr_length):
    r"""Return a permuted string from current phrase curr_phrase of length
    curr_length

    Parameters
    ----------
    curr_phrase:   str
                   temporary phrasecreated in create_phrase_candidate

    curr_length:   int
                   desired final phrase length
    Returns
    ---------
                    list
                    list of permutations
    """
    elems = curr_phrase.split()
    perms = permutations(elems, curr_length)
    return [" ".join(p) for p in perms]


def create_phrase_candidate(start_word, cands_list, curr_length):
    r"""This function creates possible candidates for the current word start_word
    Candidates are selected based on length with respect to PHRASE LENGTH and
    matching with PHRASE_COUNTER

    Parameters
    ---------
    start_word:     str
                    word which is currently screened through all the anagrams

    cands_dict:     list
                    all the plausible words which satisfy minimal length and
                    Counter requirements with start_word

    curr_length:    int
                    current number of words for the final output phrase
                    e.g. 3, 4, 5...
    Returns
    -------
                    list
                    list of permuted phrases to be hashsed
    """
    # The curr_length must be -1 because the start_word will be added
    anagrams_combs = combinations(cands_list, curr_length - 1)

    for anagram_comb in anagrams_combs:
        # Join all the pieces to create a full sentence
        curr_sentence = " ".join(anagram_comb)
        curr_sentence = " ".join([start_word, curr_sentence])
        sentence_counter = Counter(curr_sentence.replace(" ", ""))
        if sentence_counter == PHRASE_COUNTER:
            # Permutations and then hash
            for permutation in permutations_to_string(curr_sentence,
                                                      curr_length):
                yield permutation


def hash_matching(curr_phrase):
    r"""Function to match curr_phrase hash with reference HASH_X

    Parameters
    ----------
    curr_phrase:    str
                    current phrase created via permutations

    Returns
    -------
                    bool
                    True the hash matches, False does not match
    """
    curr_hash = md5(curr_phrase.encode('utf-8')).hexdigest()

    if curr_hash == HASH_EASY:
        print("{} match hash: {}".format(curr_phrase, HASH_EASY))
        return True

    elif curr_hash == HASH_MED:
        print("{} match hash: {}".format(curr_phrase, HASH_MED))
        return True

    elif curr_hash == HASH_HARD:
        print("{} match hash: {}".format(curr_phrase, HASH_HARD))
        return True
    else:
        return False


def hash_and_match(start_word, cands_list, curr_length, complete):
    r"""This function performs the search for possible candidate phrases and
    compare the hashes

    Parameters
    ---------
    start_word:     str
                    word which is currently screened through all the anagrams

    cands_dict:     list
                    all the plausible words which satisfy minimal length and
                    Counter requirements with start_word

    curr_length:    int
                    current number of words for the final output phrase
                    e.g. 3, 4, 5...

    complete:       int
                    number of hashes completed
    Returns
    -------
    complete:       int
                    updated number of hashes completed
    """
    for perm in create_phrase_candidate(start_word, cands_list, curr_length):
        matched = hash_matching(perm)
        if matched:
            # Hoooray
            complete += 1
    return complete


def run():
    r"""To crack the hash perform the following steps:

    1. Filter all words so that word Counter is a subset of ANAGRAM
    2. Scan all the anagrams and create possible candidates for each word
    3. Create a phrase for each word and combination and check requirements
    4. Permute and hash
    """
    # Initialise list of anagrams
    anagrams = []
    print("Filtering out all the unnecessary words...")
    with open('wordlist') as wordlist:
        for word in wordlist:
            # Retrieve the current word Counter
            word_letters = Counter(word.rstrip('\n'))
            # Is Counter subset of PHRASE_COUNTER?
            is_subset = is_counter_subset(PHRASE_COUNTER, word_letters)
            if is_subset:
                anagrams.append(word.strip("\n"))
    print("Found {} anagrams...".format(len(anagrams)))
    # Avoid duplicates
    anagrams = list(set(anagrams))
    # Sort all the anagrams by length
    anagrams.sort(key=len, reverse=True)
    # Define the longest anagram to know what is the minimal length
    longest_anagram = len(max(anagrams, key=len))
    # This is the minimal length we must have when coupling words
    minimal_length = PHRASE_CHARACTERS - longest_anagram
    print("Creating words combinations...")
    # When complete = 3 break the while loop
    complete = 0
    completed = False
    # Setting up an initial phrase length e.g. 3 words
    desired_length = 3
    # Set up a flag if candidate words have been already created
    candidates_flag = False
    # Set up an empy dictionary to store start_word and candidates
    cands_dict = {}

    while not completed:
        for i, word in enumerate(anagrams, 0):

            # Empty list for candidates for word
            candidates = []
            j = i+1
            # If candidate have note been scanned yet
            if not candidates_flag:
                while j < len(anagrams):
                    word2 = anagrams[j]
                    curr_phrase = word + word2
                    # Total length of pair
                    curr_phrase_len = len(curr_phrase)
                    if (curr_phrase_len >= minimal_length):
                        curr_phrase_counter = Counter(curr_phrase)
                        # Counter check
                        is_subset = is_counter_subset(PHRASE_COUNTER,
                                                      curr_phrase_counter)
                        if is_subset:
                            if word2 in candidates:
                                pass
                            else:
                                candidates.append(word2)
                    j += 1
                cands_dict[word] = candidates

            complete = hash_and_match(word,
                                      cands_dict[word],
                                      desired_length,
                                      complete)

            if complete == 3:
                completed = True
                break


        # Scanned all the words try with a longer phrase
        desired_length += 1
        candidates_flag = True
        if not completed:
            print("Updating sentence length to {}".format(desired_length))


if __name__ == '__main__':
    run()


Filtering out all the unnecessary words...
Found 1788 anagrams...
Creating words combinations...
ty outlaws printouts match hash: 23170acc097c24edb98fc5488ab033fe
printout stout yawls match hash: e4820b45d2277f3844eac66c903e84be
Updating sentence length to 4
wu lisp not statutory match hash: 665e5bcb0c20062fe8abaaf4628bb154
